In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
tour = validation_data_input.get_data('tour',config['tour_survey_dir'])
hh = validation_data_input.get_data('household',config['survey_dir'])
person = validation_data_input.get_data('person',config['survey_dir'])

In [3]:
# parcel land use data
df_parcel = pd.read_csv(os.path.join(model_dir,r'outputs\landuse\buffered_parcels.txt'),
                        delim_whitespace=True,
                        usecols=['parcelid','emptot_1','hh_1'])

In [4]:
df_tour = tour.copy()
df_hh = hh.copy()
df_person = person.copy()

# auto_ownership with 4+
df_hh['auto_ownership_4+'] = df_hh['hhvehs'].apply(lambda x: "4+" if x>=4.0 else str(x))
# hhsize with 4+
df_hh['hhsize_4+'] = df_hh['hhsize'].apply(lambda x: "4+" if x>=4.0 else str(x))

# add person type labels
ptype_cat = {1: "full time worker",
             2: "part time worker",
             3: "non-worker age 65+",
             4: "other non-working adult",
             5: "university student",
             6: "grade school student/child age 16+",
             7: "child age 5-15",
             8: "child age 0-4"}
df_person['pptyp_label'] = df_person['pptyp'].map(ptype_cat)

mode_cat = {1: "1: walk",
            2: "2: bike",
            3: "3: sov",
            4: "4: hov 2",
            5: "5: hov 3+",
            6: "6: walk to transit",
            7: "7: park-and-ride",
            8: "8: school bus",
            9: "9: other–survey only"}
df_tour['tmodetp_label'] = df_tour['tmodetp'].map(mode_cat)

df_hh = df_hh.merge(df_parcel,how="left",left_on='hhno',right_on='parcelid')
df_person = df_person.merge(df_hh,
                          how='left', on=['hhno','source']) # get auto ownership from hh data

df_tour = df_tour.merge(df_person, how='left', on=['pno','hhno','source'])

In [5]:
wk_tour = df_tour.loc[(df_tour['parent']==0) & (df_tour['pdpurp']==1)].copy()

In [6]:
df_plot = wk_tour.groupby(['source','tmodetp_label'])['toexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['toexpfac']. \
    apply(lambda x: x / float(x.sum()))
df_plot['source'] = df_plot['source'].astype(pd.CategoricalDtype(['model', 'survey']))

df_plot_ct = wk_tour.groupby(['source','tmodetp_label'])['toexpfac'].count().reset_index(). \
    rename(columns={'toexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','tmodetp_label'])

fig = px.bar(df_plot.sort_values(by=['source']), x="tmodetp_label", y="percentage", color="source",
             barmode="group",hover_data=['sample count'],title="work tour mode")
fig.update_layout(height=400, width=700, font=dict(size=11),
                  xaxis = dict(dtick = 1, categoryorder='category ascending'),
                  yaxis=dict(tickformat=".2%"))
fig.show()

### mode choice by segment

In [7]:
def plot_mode_choice(df: pd.DataFrame, grp_var: str, order_list: dict, title_name: str, n_nol: int, height=400):
    df_plot = df.groupby(['source',grp_var,'tmodetp_label'])['toexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source',grp_var], group_keys=False)['toexpfac']. \
        apply(lambda x: x / float(x.sum()))

    df_plot_ct = df.groupby(['source',grp_var,'tmodetp_label'])['toexpfac'].count().reset_index(). \
        rename(columns={'toexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source',grp_var,'tmodetp_label'])

    fig = px.bar(df_plot.sort_values(['tmodetp_label']),
                 x="percentage", y="tmodetp_label", color="source",barmode="group",
                 facet_col=grp_var, facet_col_wrap=n_nol, orientation='h',
                 hover_data=['sample count'],
                 category_orders=order_list,
                 title="work tour mode choice by " + title_name)
    fig.update_layout(height=height, width=800)
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.for_each_xaxis(lambda a: a.update(tickformat = ".1%"))
    fig.show()

In [8]:
plot_mode_choice(wk_tour,"pptyp_label",
                 {"pptyp_label":["full time worker","part time worker","non-worker age 65+","other non-working adult",
                                 "university student","grade school student/child age 16+","child age 5-15","child age 0-4"],
                  "tmodetp_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: walk to transit",
                                   "7: park-and-ride","8: school bus","9: other–survey only"]},
                 "person type",2,1000)

In [9]:
plot_mode_choice(wk_tour,"hhsize_4+",
                 {"hhsize_4+":["1","2","3","4+"],
                  "tmodetp_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: walk to transit",
                                   "7: park-and-ride","8: school bus","9: other–survey only"]},
                 "household size",2,600)

In [10]:
plot_mode_choice(wk_tour.loc[wk_tour['auto_ownership_4+']!="-1"],"auto_ownership_4+",
                 {"auto_ownership_4+":["0","1","2","3","4+"],
                  "tmodetp_label":["1: walk","2: bike","3: sov","4: hov 2","5: hov 3+","6: walk to transit",
                                   "7: park-and-ride","8: school bus","9: other–survey only"]},
                 "auto ownership",2,800)